In [49]:
%load_ext autoreload
%autoreload 2

import docker
import logging
from jtldockerctl.dctl import create_container, logger, create_novnc_container, only_one_container
import hashlib
from names_generator import generate_name
from slugify import slugify

logging.basicConfig(level=logging.ERROR)
logger.setLevel(logging.DEBUG)

config = {
    'novnc_image':  'ghcr.io/league-infrastructure/novnc-service:v0.1.1',
    'pythonapprentice_image': 'ghcr.io/league-infrastructure/league-infrastructure/docker-codeserver-python:latest'
}

local_dir="/Users/eric/proj/league-projects/curriculum/Python-Apprentice"

def unique_hash(input_string, existing_hashes, length):
    """
    Generate a hash of the specified length that is not in the list of existing hashes.

    :param input_string: The input string to hash.
    :param existing_hashes: A list of hashes to avoid.
    :param length: The desired length of the hash.
    :return: A unique hash of the specified length.
    """
    counter = 0  # To modify the input and avoid collisions
    while True:
        # Generate a SHA256 hash with a counter appended
        candidate = hashlib.sha256(f"{input_string}{counter}".encode()).hexdigest()[:length]
        if candidate not in existing_hashes:
            return candidate
        counter += 1
        
        
        

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from slugify import slugify


def create_pa_container(client, local_dir, username, vnc_id=None, reset=False):
    # Create the container
    
    containers = client.containers.list(filters={"label": f"jtl.codeserver.username={username}"}, all=True)
    
    if container := only_one_container(containers, reset):
        return container
       
    vnc_c = client.containers.get(vnc_id)   
       
    name = slugify(username)
    
       
    ipv4 = vnc_c.attrs['NetworkSettings']['Ports']['6080/tcp'][0] 
    ipaddr = 'localhost' if  ipv4['HostIp'] == '0.0.0.0' else ipv4['HostIp']
        
    vnc_url = f"http://{ipaddr}:{ipv4['HostPort']}"
       
    logger.debug(f"VNC URL: {vnc_url}")
       
    env_vars = {
        "DOMAIN": f"{name}.do.jointheleague.org",
        "PASSWORD": "code4life",
        "VSCODE_PROXY_URI": "./proxy/{{port}}",
        "DISPLAY": f"{vnc_c.name}:0",
        "VNC_URL": vnc_url
    }
    
    
    
    labels = {
        "jtl": 'true', 
        "jtl.codeserver": 'true',  
        "jtl.codeserver.username": username,

    }
    
    container = create_container(
        client,
        image=config['pythonapprentice_image'],
        env_vars=env_vars,
        name=name,
        local_dir=local_dir,
        #ports=["8080"],
        ports = {"8080/tcp": 8080},
        labels=labels,
        networks=["x11", "caddy"],
    )

    # Start the container
    container.start()

    return container

client = docker.from_env()
nvc1 = create_novnc_container(client, config, username="eric.busboom@jointheleague.org")
nvc2 = create_novnc_container(client, config, username="bob.bobson@jointheleague.org")

pa = create_pa_container(client, local_dir, username=nvc1.labels['jtl.novnc.username'], vnc_id=nvc1.id, reset=True)

nvc1.id

DEBUG:jtldockerctl.dctl:Found 1 containers
DEBUG:jtldockerctl.dctl:Container quizzical_meitner is already running
DEBUG:jtldockerctl.dctl:Found 1 containers
DEBUG:jtldockerctl.dctl:Container xenodochial_cray is already running
DEBUG:jtldockerctl.dctl:VNC URL: http://localhost:32853
INFO:jtldockerctl.dctl:Creating container from image 'ghcr.io/league-infrastructure/league-infrastructure/docker-codeserver-python:latest'...
DEBUG:jtldockerctl.dctl:Port bindings: {'8080/tcp': None}
DEBUG:jtldockerctl.dctl:Volumes: /Users/eric/proj/league-projects/curriculum/Python-Apprentice
INFO:jtldockerctl.dctl:Connecting container to network 'x11' : 81b081a8f85991990d1603f8af7c48603159f741149d68f5d44df36b8f21fc36
INFO:jtldockerctl.dctl:Connecting container to network 'caddy' : 341c37b26522cf9aaf53c4e37df9a6b94fe50cb0cc79e05d7d35d6a3f2327fb5
INFO:jtldockerctl.dctl:Container 'eric-busboom-jointheleague-org' (674bcc3a065118c658802351227c3b2fb575e92e2872510c3f8ae0bfa3b149f3) created successfully.


'aed33507db9d89b06bdc28f8727a3b77b46479da9ba63c2a754322fe44ebbb09'

In [72]:
pa.reload()
pa.attrs['NetworkSettings']['Ports']['8080/tcp'][0] 


{'HostIp': '0.0.0.0', 'HostPort': '32859'}

In [ ]:
username="eric.busboom@jointheleague.org"
containers = client.containers.list(filters={"label": f"jtl"}, all=True)

for c in containers:
    print(c.id, c.labels)
    #c.remove(force=True)    

2afcf0a1013e33189766568835054daa0bb9761a306bbcd08e31b18b1827c0eb {'dev.containers.id': 'python', 'dev.containers.release': 'v0.4.10', 'dev.containers.source': 'https://github.com/devcontainers/images', 'dev.containers.timestamp': 'Thu, 12 Dec 2024 21:37:55 GMT', 'dev.containers.variant': '3.12-bookworm', 'devcontainer.metadata': '[ {"id":"ghcr.io/devcontainers/features/common-utils:2"}, {"id":"ghcr.io/devcontainers/features/git:1"}, {"id":"ghcr.io/devcontainers/features/node:1","customizations":{"vscode":{"extensions":["dbaeumer.vscode-eslint"]}}}, {"id":"ghcr.io/devcontainers/features/python:1","customizations":{"vscode":{"extensions":["ms-python.python","ms-python.vscode-pylance","ms-python.autopep8"],"settings":{"python.defaultInterpreterPath":"/usr/local/python/current/bin/python","[python]":{"editor.defaultFormatter":"ms-python.autopep8"}}}}}, {"customizations":{"vscode":{"settings":{"python.defaultInterpreterPath":"/usr/local/bin/python"}}},"remoteUser":"vscode"} ]', 'jtl': 'true

In [47]:
nvc.kill()

In [5]:
nvc.attrs['NetworkSettings']['Networks']

{'bridge': {'IPAMConfig': None,
  'Links': None,
  'Aliases': None,
  'MacAddress': '',
  'DriverOpts': None,
  'NetworkID': '',
  'EndpointID': '',
  'Gateway': '',
  'IPAddress': '',
  'IPPrefixLen': 0,
  'IPv6Gateway': '',
  'GlobalIPv6Address': '',
  'GlobalIPv6PrefixLen': 0,
  'DNSNames': None}}

In [14]:
diffs= r.diff()
diffs

[{'Path': '/tmp', 'Kind': 0},
 {'Path': '/tmp/codeserver-stdout---supervisor-i554tzg5.log', 'Kind': 1},
 {'Path': '/tmp/foo.bar', 'Kind': 1},
 {'Path': '/tmp/vscode-git-ddaf3d0163.sock', 'Kind': 1},
 {'Path': '/tmp/vscode-ipc-a6e94b0b-66f7-4374-a7ad-4c0bbeb0dcb2.sock',
  'Kind': 1},
 {'Path': '/tmp/vscode-ipc-ef8f743e-6d70-4d82-89c3-e9c06c437117.sock',
  'Kind': 1},
 {'Path': '/tmp/codeserver-stderr---supervisor-13gnqje6.log', 'Kind': 1},
 {'Path': '/tmp/vscode-ipc-28b5f5d5-93f0-4a19-8cf9-9b64098d8589.sock',
  'Kind': 1},
 {'Path': '/tmp/vscode-ipc-5d750aeb-e98a-443e-8175-0dc27f9386d1.sock',
  'Kind': 1},
 {'Path': '/usr', 'Kind': 0},
 {'Path': '/usr/local', 'Kind': 0},
 {'Path': '/usr/local/share', 'Kind': 0},
 {'Path': '/usr/local/share/ca-certificates', 'Kind': 0},
 {'Path': '/usr/local/share/ca-certificates/orbstack-root.crt', 'Kind': 1},
 {'Path': '/etc', 'Kind': 0},
 {'Path': '/etc/ssl', 'Kind': 0},
 {'Path': '/etc/ssl/certs', 'Kind': 0},
 {'Path': '/etc/ssl/certs/orbstack-root.c

In [ ]:
# Example usage
env_variables = {
    "DOMAIN": "codeserver.do.jointheleague.org",
    "PASSWORD": "code4life",
    "VSCODE_PROXY_URI": "./proxy/{{port}}",
    "DISPLAY": ":0",
    "VNC_URL": "http://codeserver.do.jointheleague.org/vnc/"
}

client = docker.from_env()

r = create_container(
    client, 
    name='code-server', 
    image="docker-codeserver-python-devcontainer",
    local_dir="/Users/eric/proj/league-projects/curriculum/Python-Apprentice",
    env_vars=env_variables,
    ports={"8080": 8080},
    networks=["x11", "caddy"]
)
0